<a href="https://colab.research.google.com/github/rodiwaa/learnings-pocs/blob/main/notebooks/parallel_merge_sequence_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# PLAN
# - create news, weather agents
# - run them both in parallel
# - merge both results in summary
# - run root agent to run parallel, merge in sequence
# - refer https://google.github.io/adk-docs/agents/workflow-agents/parallel-agents/#full-example-parallel-web-research
# - sets up basic foundations/ bootstraps for complex workflows
# - FUTURE
# - CICD evaluations
# - delegate comms, prompts to a2a, mcp resp
# - adk web for tracing

# OG - adk-basics.ipynb

In [ ]:
! pip install google-adk google-generativeai

In [3]:
from google.adk.agents import Agent, ParallelAgent, LlmAgent, SequentialAgent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from IPython.display import display, Markdown

In [4]:
from getpass import getpass
# from google.generativeai import genai
import google.generativeai as genai
import os

api_key = getpass("enter google api_key: ")

genai.configure(api_key=api_key)

os.environ["GOOGLE_API_KEY"] = api_key

print("google api key set")

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


enter google api_key: ··········
google api key set


In [5]:
# --- Agent Definition ---

# TODO: FIXME: FUTURE: wrap all agents in a2a wrappers for smart comm

# agents
def create_weather_agent():
    """Create the friendly news fetch agent"""
    return Agent(
        name="weather",
        model="gemini-2.5-flash",
        description="Agent specialized in fetching day forecast of given place.",
        instruction="""
        You are the "Weather Forecaster" Agent 🚗 - a specialized AI assistant that creates forecast weather.

        Your Mission:
        Get weather forecast for given location. Present in friendly manner.

        Guidelines:
        1. **General Weather**: Focus on general weather conditions like hot, sunny, rainy.
        2. **Temperature**: Create morning, afternoon, and evening forecast for day.
        3. **AQI Pollution Info**: Find relevant info on pollution levels. Mention AQI levels with categories (Good, Safe, Warning, Critical).
        4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

        RETURN summary in MARKDOWN FORMAT with clear time blocks.
        """,
        tools=[google_search],
        output_key="weather-results"
    )

weather_agent = create_weather_agent()
print(f"🧞 Agent '{weather_agent.name}' is created and ready to bring you weather updates!")



🧞 Agent 'weather' is created and ready to bring you weather updates!


In [6]:
def create_news_agent():
    """Create the Spontaneous Day Trip Generator agent"""
    return Agent(
        name="day_trip_agent",
        model="gemini-2.5-flash",
        description="Agent specialized in finding news headlines for major categories.",
        instruction="""
        You are the "News" Generator 🚗 - a specialized AI assistant that brings important news for the day.

        Your Mission:
        Get top 2 headlines and new article URL for categories - World, Sports (Cricket), Business, Technology, Local (Mumbai) only.

        Guidelines:
        1. **Headlines and URL**: Make sure headlines summarise gist of news item. Provide clickable URL.
        2. **Categories**: Stick to given categories only - World, Sports (Cricket), Business, Technology, Local (Mumbai) only.
        3. **Avoid**: Avoid these topics - Politics, Bollywood.

        RETURN news in MARKDOWN FORMAT with clear healines and working URLs.
        """,
        tools=[google_search],
        output_key="news-results"
    )

news_agent = create_news_agent()
print(f"🧞 Agent '{news_agent.name}' is created and ready to bring you news!")

🧞 Agent 'day_trip_agent' is created and ready to bring you news!


In [7]:
def create_parallel_agent():
  """Parallel agent to run weather, news agents in parallel"""
  return ParallelAgent(
      name="parallel_agent_news_weather",
      # model="gemini-2.5-flash", # apparently not needed.
      description="Runs weather and news agents in parallel.",
      sub_agents=[
          weather_agent, news_agent
      ]
  )

parallel_agent = create_parallel_agent()
print(f"🧞 Agent '{parallel_agent.name}' is created and ready!")

🧞 Agent 'parallel_agent_news_weather' is created and ready!


In [11]:
def create_merger_agent_news_weather_agent():
  return LlmAgent( # LLMAgent --> Agent (alias) --> simply implements BaseAgent
     name="merger_agent_news_weather",
     model="gemini-2.5-flash",
      #TODO: FIXME: use variables for descriptions, instructions --> future MCP (prompt) servers?
     instruction="""You are an AI Assistant responsible for combining news, weather findings into a structured report.

 Your primary task is to synthesize the following results, clearly attributing findings to their source areas. Structure your response using headings for each topic. Ensure the report is coherent and integrates the key points smoothly.

 **Crucially: Your entire response MUST be grounded *exclusively* on the information provided in the 'Input Summaries' below. Do NOT add any external knowledge, facts, or details not present in these specific results.**

 **Input Summaries:**

 *   **News:**
     {news-results}

 *   **Weather:**
     {weather-results}

 **Output Format:**

 ## Summary of Recent Sustainable Technology Advancements

 ### News Results
 (Based on News Helper Agent findings)
 [Synthesize and elaborate *only* on the renewable energy input summary provided above.]

 ### Weather Results
 (Based on Weather Helper Agent findings)
 [Synthesize and elaborate *only* on the EV input summary provided above.]

 ### Overall Summary
 [Provide a brief (1-2 sentence) concluding statement that connects *only* the results presented above.]

 MARKDOWN output *only* the structured report following this format. Do not include introductory or concluding phrases outside this structure, and strictly adhere to using only the provided input summary content.
 """,
     description="Combines research findings from news, weather agents into a structured strictly grounded on provided inputs.",
     # No tools needed for merging
     # No output_key needed here, as its direct response is the final output of the sequence
 )

merger_agent_news_weather_agent = create_merger_agent_news_weather_agent()
print(f"🧞 Agent '{merger_agent_news_weather_agent.name}' is created and ready!")

🧞 Agent 'merger_agent_news_weather' is created and ready!


In [12]:
async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )
    final_response = ""
    try:
       async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):

         if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
      final_response = f"An error occurred: {e}"

    print("\n" + "-"*50)
    print("✅ Final Response:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

session_service = InMemorySessionService()
my_user_id = "rodi_adventurer_001"

In [17]:
 # --- 4. Create the SequentialAgent (Orchestrates the overall flow) ---
 # This is the main agent that will be run. It first executes the ParallelAgent
 # to populate the state, and then executes the MergerAgent to produce the final output.
def create_root_agent():
  return SequentialAgent(
     name="NewsAndWeatherPipeline",
     # Run parallel research first, then merge
     sub_agents=[parallel_agent, merger_agent_news_weather_agent],
     description="Coordinates parallel news and weather the results."
 )

root_agent = create_root_agent()
print(f"🧞 Agent '{root_agent.name}' is created and ready!")


ValidationError: 1 validation error for SequentialAgent
  Value error, Agent `parallel_agent_news_weather` already has a parent agent, current parent: `NewsAndWeatherPipeline`, trying to add: `NewsAndWeatherPipeline` [type=value_error, input_value={'name': 'NewsAndWeatherP...d weather the results.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

In [19]:
async def run_root_agent():
    # Create a new, single-use session for this query
    root_session = await session_service.create_session(
        app_name=root_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Find me top news headlines and weather updates for the day around Mumbai"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(root_agent, query, root_session, my_user_id)

await run_root_agent()

🗣️ User Query: 'Find me top news headlines and weather updates for the day around Mumbai'

🚀 Running query for agent: 'NewsAndWeatherPipeline' in session: '95713105-bd72-48d4-b5cf-2820927c6906'...

--------------------------------------------------
✅ Final Response:


## Summary of Recent Sustainable Technology Advancements

### News Results
(Based on RenewableEnergyResearcher's findings)
Mumbai is set to experience a 10% water cut in the Island City and Eastern Suburbs for 11 days, commencing on January 27. Additionally, a new flyover connecting Dindoshi Court and Film City is anticipated to open for traffic by the end of May.

### Weather Results
(Based on EVResearcher's findings)
Mumbai can anticipate a clear and sunny day. The morning will begin cool, around 67°F (19°C), with potential mist or haze, before warming to 75-78°F (24-25.5°C). The afternoon promises bright sunshine, with temperatures peaking between 78-82°F (25.5-28°C). The evening will be clear and pleasant, with temperatures gradually cooling to approximately 69-72°F (20.5-22°C).

The current Air Quality Index (AQI) in Mumbai ranges from 148 to 166, placing it in the 'Unhealthy for Sensitive Groups' to 'Unhealthy' categories. Individuals with respiratory conditions, children, and the elderly may experience health effects and are advised to limit prolonged outdoor exertion. The general public is also recommended to consider limiting prolonged outdoor activity.

Regarding local news, Mumbai, along with 14 other cities, is slated to have women mayors, amidst allegations of rigging by Sena (UBT). Infrastructure development includes a new flyover connecting Dindoshi Court and Film City, expected to open for traffic by the end of May. Parts of Thane will face a 12-day phased water cut starting January 27th. Economically, the Chief Minister announced investment commitments worth ₹30 lakh crore secured at the World Economic Forum (WEF) in Davos. Furthermore, a push-pull Amrit Bharat train rake has arrived in Mumbai for high-speed trials at 130 km/h.

### Overall Summary
Mumbai is projected to have a clear and sunny day, though with an 'Unhealthy' Air Quality Index for sensitive groups, while also facing a 10% water cut in some areas and seeing progress in local infrastructure and economic commitments.

--------------------------------------------------

